In [1]:
import torch, time, os
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
import pandas as pd
import matplotlib.pyplot as plt
from models import *
from Losses import *
from Metrics import evaluate, cross_bound_check
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split

# class TimeSeriesDataLoader:
#     def __init__(self, data, window_size, label_column, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, scaler=None):
#         self.data         = data
#         self.window_size  = window_size
#         self.label_column = label_column
#         self.train_ratio  = train_ratio
#         self.val_ratio    = val_ratio
#         self.test_ratio   = test_ratio
#         self.scaler       = scaler

#         self.train_X, self.train_y = None, None
#         self.val_X, self.val_y = None, None
#         self.test_X, self.test_y = None, None

#         self._prepare_data()

#     def _prepare_data(self):
#         # 删除缺失值
#         self.data = self.data.dropna()

#         # 划分特征和目标变量
#         X = self.data.drop(columns=[self.label_column]).values
#         y = self.data[self.label_column].values

#         # 处理缺失值（如果有）
#         # 进行标准化（如果需要）
#         if self.scaler is None:
#             self.scaler_x = StandardScaler()
#             self.scaler_y = StandardScaler()
#             X = self.scaler_x.fit_transform(X)
#             y = self.scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

#         # 生成窗口数据
#         sequences_X, sequences_y = [], []
#         for i in range(len(self.data) - self.window_size + 1):
#             window_X = X[i:i + self.window_size]
#             window_y = y[i + self.window_size - 1]  # 取窗口最后一个样本作为目标变量
#             sequences_X.append(window_X)
#             sequences_y.append(window_y)
#         sequences_X, sequences_y = np.array(sequences_X), np.array(sequences_y)
#         sequences_X = sequences_X.reshape(len(sequences_X), self.window_size*sequences_X.shape[2])

#         # 划分训练集、验证集、测试集
#         train_size = int(len(sequences_X) * self.train_ratio)
#         val_size = int(len(sequences_X) * self.val_ratio)
#         test_size = len(sequences_X) - train_size - val_size

#         self.train_X, self.train_y = sequences_X[:train_size], sequences_y[:train_size]
#         self.val_X, self.val_y = sequences_X[train_size:train_size + val_size], sequences_y[train_size:train_size + val_size]
#         self.test_X, self.test_y = sequences_X[train_size + val_size:], sequences_y[train_size + val_size:]

#     def inverse_transform(self, data, is_label=False):
#         # 将标准化后的数据还原
#         if is_label:
#             return self.scaler_y.inverse_transform(data.reshape(-1, 1)).flatten()
#             # return self.scaler_y.inverse_transform(data)[:, 0]
#         else:
#             return self.scaler_x.inverse_transform(data)

#     def get_train_data(self, to_tensor=False):
#         if to_tensor:
#             return torch.from_numpy(self.train_X).to(torch.float32), torch.from_numpy(self.train_y).to(torch.float32)
#         else:
#             return self.train_X, self.train_y

#     def get_val_data(self, to_tensor=False):
#         if to_tensor:
#             return torch.from_numpy(self.val_X).to(torch.float32), torch.from_numpy(self.val_y).to(torch.float32)
#         else:
#             return self.val_X, self.val_y

#     def get_test_data(self, to_tensor=False):
#         if to_tensor:
#             return torch.from_numpy(self.test_X).to(torch.float32), torch.from_numpy(self.test_y).to(torch.float32)
#         else:
#             return self.test_X, self.test_y


In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

class TimeSeriesDataLoader:
    def __init__(self, data, window_size, label_column, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, scaler=None):
        self.data         = data
        self.window_size  = window_size
        self.label_column = label_column
        self.train_ratio  = train_ratio
        self.val_ratio    = val_ratio
        self.test_ratio   = test_ratio
        self.scaler       = scaler

        self.train_X, self.train_y = None, None
        self.val_X, self.val_y = None, None
        self.test_X, self.test_y = None, None

        self._prepare_data()

    def _prepare_data(self):
        # 删除缺失值
        self.data = self.data.dropna()

        # 划分特征和目标变量
        X = self.data.drop(columns=[self.label_column]).values
        y = self.data[self.label_column].values

        # 处理缺失值（如果有）
        # 进行标准化（如果需要）
        if self.scaler is None:
            self.scaler_x = StandardScaler()
            self.scaler_y = StandardScaler()
            X = self.scaler_x.fit_transform(X)
            y = self.scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

        data = np.concatenate((X, y.reshape(-1, 1)), axis=1)
        # 生成窗口数据
        sequences_X, sequences_y = [], []
        for i in range(len(self.data) - self.window_size + 1):
            window_X = data[i:i + self.window_size]
            window_y = y[i + self.window_size - 1]  # 取窗口最后一个样本作为目标变量
            sequences_X.append(window_X)
            sequences_y.append(window_y)
        sequences_X, sequences_y = np.array(sequences_X), np.array(sequences_y)
        # print(f'x: {sequences_X.shape}, y: {sequences_y.shape}')
        # sequences_X = np.concatenate((sequences_X, sequences_y.reshape(-1, 1)), axis=1)
        sequences_X = sequences_X.reshape(len(sequences_X), self.window_size*sequences_X.shape[2])

        # 划分训练集、验证集、测试集
        train_size = int(len(sequences_X) * self.train_ratio)
        val_size = int(len(sequences_X) * self.val_ratio)
        test_size = len(sequences_X) - train_size - val_size

        self.train_X, self.train_y = sequences_X[:train_size], sequences_y[:train_size]
        self.val_X, self.val_y = sequences_X[train_size:train_size + val_size], sequences_y[train_size:train_size + val_size]
        self.test_X, self.test_y = sequences_X[train_size + val_size:], sequences_y[train_size + val_size:]

    def inverse_transform(self, data, is_label=True):
        # 将标准化后的数据还原
        if is_label:
            if data.ndim < 2:
                return self.scaler_y.inverse_transform(data.reshape(-1, 1)).flatten()
            else:
                for i in range(data.shape[1]):
                    data[:, i] = self.scaler_y.inverse_transform(data[:, i].reshape(-1, 1)).flatten()
                return data
        else:
            return self.scaler_x.inverse_transform(data)

    def get_train_data(self, to_tensor=False):
        if to_tensor:
            return torch.from_numpy(self.train_X).to(torch.float32), torch.from_numpy(self.train_y).to(torch.float32)
        else:
            return self.train_X, self.train_y

    def get_val_data(self, to_tensor=False):
        if to_tensor:
            return torch.from_numpy(self.val_X).to(torch.float32), torch.from_numpy(self.val_y).to(torch.float32)
        else:
            return self.val_X, self.val_y

    def get_test_data(self, to_tensor=False):
        if to_tensor:
            return torch.from_numpy(self.test_X).to(torch.float32), torch.from_numpy(self.test_y).to(torch.float32)
        else:
            return self.test_X, self.test_y


In [4]:
data_path = '../data/Kaggle Wind Power Forecasting Data/Turbine_Data.csv'
df        = pd.read_csv(data_path, parse_dates=["Unnamed: 0"])
df        = df[['ActivePower', 'WindDirection','WindSpeed']]
df.dropna(axis=0, inplace=True)
df['WindDirection_sin'] = np.sin(df['WindDirection'])
df['WindDirection_cos'] = np.cos(df['WindDirection'])
df.drop('WindDirection', axis=1, inplace=True)
df.head()

,ActivePower,WindSpeed,WindDirection_sin,WindDirection_cos
144,-5.357727,2.279088,0.989358,-0.145500
145,-5.822360,2.339343,-0.918521,0.395372
146,-5.279409,2.455610,0.650311,0.759668
147,-4.648054,2.026754,-0.543996,0.839088
148,-4.684632,1.831420,-0.543996,0.839088


In [5]:
df = df.iloc[:2000]
df.describe()

,ActivePower,WindSpeed,WindDirection_sin,WindDirection_cos
count,2000.000000,2000.000000,2000.000000,2000.000000
mean,360.848293,4.902579,0.030517,0.000677
std,385.623285,1.710056,0.591279,0.806199
min,-10.476446,1.306637,-0.999981,-0.999999
25%,48.966478,3.530923,-0.387809,-0.899880
50%,237.797081,4.765032,0.017672,0.030945
75%,545.578402,6.186512,0.510164,0.895984
max,1713.238386,9.078578,0.999998,1.000000


In [6]:
window_size = 2
label_column = 'ActivePower'

loader = TimeSeriesDataLoader(df, window_size, label_column)
X_train, Y_train = loader.get_train_data()
X_val  , Y_val   = loader.get_val_data()
X_test , Y_test  = loader.get_test_data()

print(X_train.shape, Y_train.shape)
print(X_val.shape, Y_val.shape)
print(X_test.shape, Y_test.shape)

(1399, 8) (1399,)
(299, 8) (299,)
(301, 8) (301,)


In [7]:
from models import QuantileRegressionEstimator
from Losses import PinballLoss

class NESCQR:
    def __init__(self, data_loader, model_pool:list, label_pool:list, batch_size:int, M:int, alpha_set:list, 
                 l_rate:float, max_epochs:int, replace, symmetric, saveflag, save_dir, 
                 alpha_base=None, step=2, device='cuda', verbose=True):
        assert 0 < M <= len(model_pool), "M must be in range (0, len(model_pool)]"
        self.data_loader = data_loader
        self.model_pool  = model_pool
        self.label_pool  = label_pool  # 与model_pool里每个模型一一对应的模型名字
        self.batch_size  = batch_size
        self.M           = M           # 最终的集成模型的基学习器个数
        self.alpha_set   = alpha_set   # 置信水平集合
        self.l_rate      = l_rate      # 学习率
        self.max_epochs  = max_epochs
        self.device      = device
        self.saveflag   = saveflag
        self.save_dir    = save_dir
        self.alpha_base  = alpha_base if alpha_base else max(alpha_set)
        self.quantiles   = [self.alpha_base / 2, 1 - self.alpha_base / 2]
        self.loss_fn     = PinballLoss(self.quantiles, self.device)
        self.replace     = replace    # 是否有放回地前向选择
        self.step        = step       # DMCQR算法更新步长，int, 越小更新越快越准确
        self.symmetric   = symmetric  # 是否采用对称性conformity score
        # self.logger      = logger
        self.verbose     = verbose
        
    def init_training(self, saveflag=False):
        # 先训练好每个基学习器
        X_train, Y_train = self.data_loader.get_train_data(to_tensor=True)
        X_val  , Y_val   = self.data_loader.get_val_data(to_tensor=True)
        # X_train, Y_train = X_train.to(self.device), Y_train.to(self.device)
        # X_val  , Y_val   = X_val.to(self.device), Y_val.to(self.device)

        assert len(X_train) == len(Y_train)
        assert len(X_val)   == len(Y_val)
        assert len(X_test)  == len(Y_test)

        print(f'X_train.shape: {X_train.shape}, Y_train.shape: {Y_train.shape}')
        print(f'X_val.shape: {X_val.shape}, Y_val.shape: {Y_val.shape}')
        # print(f'X_test.shape: {X_test.shape}, Y_train.shape: {Y_test.shape}')

        num_models = len(self.model_pool)
        model_pool_trained = []
        for i, model in enumerate(self.model_pool):
            print(f'Model {i+1}/{num_models} {self.label_pool[i]} starts training...')

            # 采用DMCQR得到最终的预测区间，则只需要最大的alpha，即两条分位数即可得到多条预测区间上下界。
            learner = QuantileRegressionEstimator(model, [max(self.alpha_set)], self.max_epochs,
                                                   self.batch_size,self.device, self.l_rate, self.verbose)
            learner.fit(X_train, Y_train, X_val, Y_val)
            model_pool_trained.append(learner)
            print(f'Model {i+1}/{num_models} {self.label_pool[i]} finished training.')
            
            if saveflag:
                torch.save(learner, f'{self.save_dir}/trained_models/{self.label_pool[i]}.pth')
                print(f'Model {i+1}/{num_models} saved.')

        return model_pool_trained

    def forward_selection(self, model_pool_trained, label_pool, replace=True):
        # 前向选择出最优集成模型组合
        X_val  , Y_val   = self.data_loader.get_val_data(to_tensor=True)
        X_val  , Y_val   = X_val.to(self.device), Y_val.to(self.device)
        # pool = dict(zip(label_pool, model_pool_trained))
        if replace:
            print('Forward selection with replacement.')
        else:
            print('Forward selection without replacement.')

        selected_model, selected_label = [], []
        while len(selected_model) < self.M:
            best_loss = np.inf
            
            for i in range(len(model_pool_trained)):
                models_ = selected_model + [model_pool_trained[i]]
                merged_output = torch.stack([torch.from_numpy(model.predict(X_val)) for model in models_])
                merged_output = torch.mean(merged_output, axis=0)
                loss = self.loss_fn(merged_output, Y_val)
                if loss.item() < best_loss:
                    best_model = model_pool_trained[i]
                    best_loss  = loss.item()
                    best_label = i

            selected_model.append(best_model)
            selected_label.append(label_pool[best_label])
            if not replace:  # 无放回
                model_pool_trained.pop(best_label)
                
        print(f'Model selected: {selected_label}')

        return selected_model, selected_label

    def conformal(self, res_val, Y_val, res_test, Y_test, step, symmetric=True):
        # DMCQR
        assert res_val.shape[0] == Y_val.shape[0]
        assert res_val.shape[1] == 2
        assert res_test.shape[1] == 2

        Y_val  , Y_test   = np.array(Y_val),   np.array(Y_test)
        res_val, res_test = np.array(res_val), np.array(res_test)
        Y_all     = np.concatenate((Y_val, Y_test), axis=0)
        res_all   = np.concatenate((res_val, res_test), axis=0)
        num_alpha = len(self.alpha_set)
        conf_PI   = np.zeros((len(res_test), num_alpha*2))
        val_size  = len(Y_val)
        test_size = len(Y_test)

        if symmetric:  
            # DMCQRS, use symmetric conformity score, 对称误差集合
            print('Use symmetric conformity score to calibrate quantiles.')
            E = list(np.max((res_val[:, 0] - Y_val, Y_val - res_val[:, -1]), axis=0))  # 误差集合，队列，先进先出
            Q = np.zeros(num_alpha)

            for t in range(val_size, val_size + test_size):
                for i, alpha in enumerate(self.alpha_set):
                    Q[i] = np.quantile(E, (1-alpha)*(1+1/val_size))
                    conf_PI[:, i] = res_test[:, 0] - Q[i]
                    conf_PI[:, -(i+1)] = res_test[:, -1] + Q[i]

                    if t % step == 0:
                        # print(f't = {t}, Q = {Q}')
                        for j in range(t - self.step, t - 1):
                            e = np.max((res_all[j, 0] - Y_all[j], Y_all[j] - res_all[j, -1]),axis=0)
                            E.pop(0)
                            E.append(e)   

            return conf_PI
        
        else:   
            # DMCQRS, use asymmetric conformity score, 非对称误差集合
            print('Use asymmetric conformity score to calibrate quantiles.')
            Q_low, Q_high = np.zeros(num_alpha), np.zeros(num_alpha)
            E_low = list(res_val[:, 0] - Y_val)    # 下界误差集合
            E_high = list(Y_val - res_val[:,-1])   # 上界误差集合
                
            for t in range(val_size, val_size + test_size):
                for i, alpha in enumerate(self.alpha_set):
                    Q_low[i] = np.quantile(E_low, (1 - alpha / 2))
                    Q_high[-(i+1)] = np.quantile(E_high, (1 - alpha / 2))
                    
                    conf_PI[:, i] = res_test[:, 0] - Q_low[i]
                    conf_PI[:, -(i+1)] = res_test[:, -1] + Q_high[-(i+1)]

                if t % step == 0:
                    # print(f't: {t}, Q_low: {Q_low}, Q_high: {Q_high}')
                    for j in range(t - step, t - 1):
                        e_low = res_all[j, 0] - Y_all[j]
                        e_high = Y_all[j] - res_all[j, -1]
                        E_low.pop(0)
                        E_low.append(e_low)
                        E_high.pop(0)
                        E_high.append(e_high)

            return conf_PI     

    def fit(self):
        model_pool_trained = self.init_training()
        self.model_pool_selected, self.selected_label = self.forward_selection(model_pool_trained, self.label_pool, self.replace)

    def predict(self, X_test=None, Y_test=None, inverse_normalization=True):
        # construct prediction intervals
        X_val  , Y_val   = self.data_loader.get_val_data(to_tensor=True)
        if not X_test:
            X_test,  Y_test  = self.data_loader.get_test_data(to_tensor=True)
        Y_val  , Y_test  = Y_val.detach().numpy(), Y_test.detach().numpy()
        X_val   = X_val.to(self.device)
        X_test  = X_test.to(self.device)

        pred = self.model_pool_selected[0].predict(X_val)
        # print(f'pred.shape: {pred.shape}')
        res_val = torch.stack([torch.from_numpy(model.predict(X_val)) for model in self.model_pool_selected])
        res_val = torch.mean(res_val, axis=0)
        res_val = res_val.detach().numpy()
        res_test = torch.stack([torch.from_numpy(model.predict(X_test)) for model in self.model_pool_selected])
        res_test = torch.mean(res_test, axis=0)
        res_test = res_test.detach().numpy()
        # print(f'res_val.shape: {res_val.shape}, res_test.shape: {res_test.shape}')

        self.conf_PI = self.conformal(res_val, Y_val, res_test, Y_test, self.step, self.symmetric)
        if inverse_normalization:  # 逆标准化回原来的量纲
            self.conf_PI = self.data_loader.inverse_transform(self.conf_PI, is_label=True)

        cols = [str(round(alpha/2, 3)) for alpha in self.alpha_set] + [str(round(1-alpha/2, 3)) for alpha in self.alpha_set]
        if self.saveflag:
            df = pd.DataFrame(self.conf_PI, columns=cols)
            df.to_csv(os.path.join(self.save_dir,'conf_PIs.csv'), index=False)
            
        return self.conf_PI

In [8]:
# 获取当前时间戳
current_time = time.strftime("%Y_%m_%d_%H_%M_%S")
# 构建文件夹路径
save_dir = os.path.join("result", current_time)
print(save_dir)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

n_ensemble = 3  #number of baseline models, which is M in paper.
M          = n_ensemble
max_epochs = 500
l_rate     = 1e-4
activation = 'tanh'  #nn.Tanh,              nn.ReLU, nn.Sigmoid
batch_size = 512
dropout    = 0.2
replace    = False
symmetric  = True
saveflag   = True
# save_dir   = './results/'
step       = 2
device     = 'cuda'
verbose    = True

alpha_set = np.array([0.05, 0.10, 0.15])
num_alpha = len(alpha_set)
alpha_base = max(alpha_set)
quantiles = [max(alpha_set)/2, 1 - max(alpha_set)/2]
# quantiles = np.zeros(2*num_alpha)
# for i in range(num_alpha):
#     quantiles[i] = alpha_set[i] / 2
#     quantiles[-(i+1)] = 1 - alpha_set[i] / 2

# loss_fn = PinballLoss(quantiles=quantiles, device=device)
input_dim = X_train.shape[1]
x_size = len(df.columns)
out_dim = len(quantiles)
kernel_size = 2
num_repeat = 1
hidden_units = [20 + i*4 for i in range(num_repeat)]
channel_sizes = [3 + i*2 for i in range(num_repeat)]

model_pool = [NET(input_dim, h, out_dim, activation) for h in hidden_units] + \
             [RNN(input_dim, h, out_dim, activation, device) for h in hidden_units] + \
             [LSTM(input_dim, h, out_dim, device) for h in hidden_units] + \
             [GRU(x_size, h, out_dim, device) for h in hidden_units] + \
             [TCN(x_size, out_dim, [c]*2, kernel_size, dropout) for c in channel_sizes]
             
# label_pool = ['BPNN']*num_repeat + ['RNN']*num_repeat + ['LSTM']*num_repeat + ['GRU']*num_repeat + ['TCN']*num_repeat
label_pool = [f'BPNN_{h}' for h in hidden_units] + \
             [f'RNN_{h}' for h in hidden_units] + \
             [f'LSTM_{h}' for h in hidden_units] + \
             [f'GRU_{h}' for h in hidden_units] + \
             [f'TCN_{c}' for c in channel_sizes]

nescqr = NESCQR(loader, model_pool, label_pool, batch_size, M, alpha_set, 
                 l_rate, max_epochs, replace, symmetric, saveflag, save_dir, alpha_base, step, 
                  device, verbose)

result\2023_10_08_18_00_23


f:\Postgraduate\papers\NESCQR\code\NESCQR\venv\lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [9]:
# X_test , Y_test  = loader.get_test_data(to_tensor=True)
nescqr.fit()
conf_PI = nescqr.predict()
conf_PI.shape

X_train.shape: torch.Size([1399, 8]), Y_train.shape: torch.Size([1399])
X_val.shape: torch.Size([299, 8]), Y_val.shape: torch.Size([299])
Model 1/5 BPNN_20 starts training...


Epoch:100, train_loss: 0.3112, validation_loss: 0.3040, cost_time: 0.03s
Epoch:200, train_loss: 0.2083, validation_loss: 0.1849, cost_time: 0.03s
Epoch:300, train_loss: 0.1564, validation_loss: 0.1211, cost_time: 0.04s
Epoch:400, train_loss: 0.1218, validation_loss: 0.0952, cost_time: 0.03s
Epoch:500, train_loss: 0.1043, validation_loss: 0.0828, cost_time: 0.03s
Model 1/5 BPNN_20 finished training.
Model 2/5 RNN_20 starts training...
Epoch:100, train_loss: 0.3388, validation_loss: 0.3471, cost_time: 0.04s
Epoch:200, train_loss: 0.2279, validation_loss: 0.2486, cost_time: 0.03s
Epoch:300, train_loss: 0.1476, validation_loss: 0.1736, cost_time: 0.03s
Epoch:400, train_loss: 0.1012, validation_loss: 0.1198, cost_time: 0.03s
Epoch:500, train_loss: 0.0766, validation_loss: 0.0891, cost_time: 0.04s
Model 2/5 RNN_20 finished training.
Model 3/5 LSTM_20 starts training...
Epoch:100, train_loss: 0.3838, validation_loss: 0.3759, cost_time: 0.03s
Epoch:200, train_loss: 0.2986, validation_loss: 0.2

(301, 6)

In [10]:
conf_PI[0]

array([237.34871709, 250.13931448, 271.58052059, 728.84945852,
       750.29067612, 763.08126202])

In [17]:
# from Metrics import *

# def evaluate(y_test, PIs, alpha_set, ita=0.5, saveflag=False, save_dir=None, verbose=True):
#     """
#     对所有预测区间进行评估。

#     Args:
#     y_test: 测试集的y, the ground truth.
#     PIs: 预测区间，从左到右依次增大，prediction intervals, ndarray.
#     alpha_set: 置信水平集合, the set of alphas.
#     ita: CWC中的一个参数，控制惩罚项的大小，注意这里的PICP和PINC均乘了100，所以这个参数不需要太大。
#     verbose: 是否输出详细信息。
    
#     out:
#     df: 包括回归和区间的评估的结果, DataFrame
    
#     """
#     assert PIs.shape[1] == len(alpha_set) * 2
#     y_test = np.array(y_test)
#     if y_test.ndim > 1:
#         y_test = y_test.squeeze()
    
#     result = []
#     for i, alpha in enumerate(alpha_set):
#         y_lower, y_upper = PIs[:, i], PIs[:, -(i+1)]
#         y_pred = (y_lower + y_upper) / 2   # Regression prediction
#         PINC = (1 - alpha) * 100
    
#         # Regression
#         MSE = mean_squared_error(y_pred, y_test)
#         MAE = mean_absolute_error(y_pred, y_test)
#         RMSE = np.sqrt(MSE)
#         CRPS = crps(y_pred, y_test)
#         SDE = sde(y_pred, y_test)

#         # Interval prediction
#         in_the_range = np.sum((y_test >= y_lower) & (y_test <= y_upper))
#         PICP = in_the_range / len(y_test) * 100
#         MPIW  = np.mean(abs(y_upper - y_lower))
#         PINAW = MPIW / (y_test.max() - y_test.min())
#         F = 2 * PICP * (1/MPIW) / (PICP + 1 / MPIW)
#         ACE = PICP - PINC
#         gamma = 1 if ACE < 0 else 0
#         CWC = PINAW * (1 + gamma * np.exp(-ita*ACE))
#         score = interval_score(y_test, y_lower, y_upper, alpha)

#         if verbose:
#             print('PINC: {:.0f}%, MAE: {:.4f}, MSE: {:.4f}, RMSE: {:.4f}, CRPS: {:.4f}, SDE: {:.4f}'.format(PINC, MAE,
#                                             MSE, RMSE, CRPS, SDE))
#             print('PINC: {:.0f}%, PICP: {:.4f}, MPIW: {:.4f}, PINAW: {:.4f}, F: {:.4f}, ACE: {:.4f}, CWC: {:.4f}, \
#                   interval_score: {:.4f}'.format(PINC, PICP, MPIW, PINAW, F, ACE, CWC, score))
#         result.append([PINC, MAE, MSE, RMSE, CRPS, SDE, PICP, MPIW, F, PINAW, ACE, CWC, score])

#     result_df = pd.DataFrame(result, columns=['PINC','MAE', 'MSE', 'RMSE', 'CRPS', 'SDE', 'PICP', 'MPIW', 'F', 'PINAW', 'ACE', 'CWC', 'Interval score'])
#     if saveflag:
#         result_df.to_csv(os.path.join(save_dir, 'metrics.csv'))
#         print('Metrics are saved to {}'.format(os.path.join(save_dir, 'metrics.csv')))
#         return result_df
#     else:
#         return result_df


In [29]:
Y_test_original = loader.inverse_transform(Y_test)
res = evaluate(Y_test_original, conf_PI, alpha_set)
res

PINC: 95%, MAE: 128.4229, MSE: 22873.5461, RMSE: 151.2400, CRPS: 128.4229, SDE: 144.5217
PINC: 95%, PICP: 95.6811, MPIW: 486.6723, PINAW: 0.4191, F: 0.0041, ACE: 0.6811, CWC: 0.4191,                   interval_score: 543.6325
PINC: 90%, MAE: 128.4229, MSE: 22873.5450, RMSE: 151.2400, CRPS: 128.4229, SDE: 144.5217
PINC: 90%, PICP: 93.6877, MPIW: 461.0912, PINAW: 0.3970, F: 0.0043, ACE: 3.6877, CWC: 0.3970,                   interval_score: 502.9226
PINC: 85%, MAE: 128.4229, MSE: 22873.5452, RMSE: 151.2400, CRPS: 128.4229, SDE: 144.5217
PINC: 85%, PICP: 88.7043, MPIW: 418.2087, PINAW: 0.3601, F: 0.0048, ACE: 3.7043, CWC: 0.3601,                   interval_score: 472.1707


,PINC,MAE,MSE,RMSE,CRPS,SDE,PICP,MPIW,F,PINAW,ACE,CWC,Interval score
0,95.0,128.422940,22873.546132,151.240028,128.422940,144.521674,95.681063,486.672334,0.004109,0.419051,0.681063,0.419051,543.632499
1,90.0,128.422936,22873.544981,151.240024,128.422936,144.521670,93.687708,461.091150,0.004337,0.397025,3.687708,0.397025,502.922608
2,85.0,128.422938,22873.545200,151.240025,128.422938,144.521670,88.704319,418.208737,0.004782,0.360101,3.704319,0.360101,472.170743


In [19]:
res_cross = cross_bound_check(conf_PI)
res_cross

r = 0, l1 = 0.0000, u1 = 0.0000
r = 1, l1 = 0.0000, u1 = 0.0000
No cross-bound phenomenon.
MUCW = 0.0000, MLCW = 0.0000
Cross loss:  0.0


,MUCW,MLCW,Cross loss
0,0,0,0.0
